# Next Word Prediction

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense

In [2]:
with open('input.txt','r',encoding="utf-8") as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_word = len(tokenizer.word_index) + 1

In [4]:
input_sequence = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequence.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequence])
input_sequence = np.array(pad_sequences(input_sequence,maxlen=max_sequence_len,padding='pre'))

In [6]:
X = input_sequence[:,:-1]
y = input_sequence[:,-1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y,num_classes=total_word))

In [8]:
model = Sequential()
model.add(Embedding(total_word,100,input_length = max_sequence_len - 1))
model.add(LSTM(150))
model.add(Dense(total_word,activation = 'softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=100,verbose=1)


Epoch 1/100


3010/3010 [==============================] - 130s 42ms/step - loss: 6.2396 - accuracy: 0.0775
Epoch 2/100
3010/3010 [==============================] - 147s 49ms/step - loss: 5.5105 - accuracy: 0.1243
Epoch 3/100
3010/3010 [==============================] - 150s 50ms/step - loss: 5.1247 - accuracy: 0.1478
Epoch 4/100
3010/3010 [==============================] - 148s 49ms/step - loss: 4.8001 - accuracy: 0.1646
Epoch 5/100
3010/3010 [==============================] - 160s 53ms/step - loss: 4.5003 - accuracy: 0.1810
Epoch 6/100
3010/3010 [==============================] - 129s 43ms/step - loss: 4.2185 - accuracy: 0.2015
Epoch 7/100
3010/3010 [==============================] - 140s 47ms/step - loss: 3.9498 - accuracy: 0.2269
Epoch 8/100
3010/3010 [==============================] - 4039s 1s/step - loss: 3.6934 - accuracy: 0.2567
Epoch 9/100
3010/3010 [==============================] - 58s 19ms/step - loss: 3.4499 - accuracy: 0.2895
Epoch 10/100
3010/3010 [=====================

In [10]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list],maxlen = max_sequence_len -1,padding = "pre")
    predicted = np.argmax(model.predict(token_list),axis = -1)
    output_word = ""
    for word,index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 37ms/step
I will leave if they will but he
